In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score ## To compute the auc score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

from sklearn.preprocessing import OneHotEncoder

## Add cross validation

In [38]:
## FOR NORMALIZATION
data = pd.read_csv('train_2008.csv')
y_train = data.pop('target').values
X_train = preprocessing.scale(np.array(data))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [9]:
from sklearn import model_selection

def cross_val(clf, X_train, y_train):
    
    result = model_selection.cross_validate(clf, X_train, y_train, scoring='roc_auc', cv=10, return_train_score=True)
    return result

In [46]:
def process_for_submission(y_pred, title):
    lst = y_pred
    print(len(lst))
    
    indexes = [i for i in range(len(lst))]
    result = []
    result.append(indexes)
    result.append(lst)
    result = np.array(result)
    result = np.transpose(result)

    
    
    #df = df.astype(int)
    df = pd.DataFrame(result, columns=['id', 'target'])
    df["id"] = df["id"].astype("int")
    df["target"] = df["target"].astype("float")
    df = df.to_csv (title + 'submission.csv', index = None, header=True)

    
    return df
    

In [93]:
process_for_submission(gbm.predict_proba(X_real)[:, 1])

16000
16000


In [14]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_161"; Java(TM) SE Runtime Environment (build 1.8.0_161-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmpmjwbouc7
  JVM stdout: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmpmjwbouc7/h2o_pavanchitta_started_from_python.out
  JVM stderr: /var/folders/cv/xqmn6z1x6l39r_575wkj9b780000gn/T/tmpmjwbouc7/h2o_pavanchitta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,"14 days, 17 hours and 51 minutes"
H2O cluster name:,H2O_from_python_pavanchitta_rsskrp
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [124]:
def drop_columns(data):
    drop_columns = []
    for column in data.columns:
        if column != 'target' and data[column].nunique() < 2:
            drop_columns.append(column)
        elif (data[column] == -1).sum() / data.shape[0] > 0.95:
            drop_columns.append(column)
    return data.drop(columns=drop_columns), drop_columns

# To be performed after drop columns is called!!!
def one_hot_encoding(data, categoricals):
    categorical_idx = []
    count = 0
    for column in categoricals:
        if column in data and data[column].nunique() < 10  and (data[column] < 0).sum() == 0:
            count += 1
            #print(data.count_vals)
            categorical_idx.append(data.columns.get_loc(column))
    print(count)
    enc = OneHotEncoder(categorical_features=categorical_idx)
    return enc.fit_transform(data).toarray(), categorical_idx
    

In [125]:
data = pd.read_csv('train_2008.csv')
data, drop_columns = drop_columns(data)
data = data.drop(columns=['id'])
y_train = data.pop('target').values

## Add one to each of the values in the dataframe in order to get rid of -1 values
data += 1

categoricals =['HURESPLI', 'HUFINAL', 'HUSPNISH', 'HETENURE', 'HEHOUSUT', \
               'HETELHHD', 'HETELAVL', 'HEPHONEO', 'HUTYPEA', 'HUTYPB', 'HUTYPC', \
               'HRINTSTA', 'HRHTYPE', 'HUINTTYP', 'HRLONGLK', 'HUBUS', 'GEREG', \
               'GESTCEN', 'GESTFIPS', 'GTCBSA', 'GTCO', 'GTCBSAST', 'GTMETSTA', \
               'PUPELIG', 'PERRP', 'PRTFAGE', 'PEMARITL', 'PESEX', 'PEAFEVER', \
               'PEAFNOW', 'PEEDUCA', 'PTDTRACE', 'PRDTHSP', 'PUCHINHH', 'PRFAMNUM', \
               'PRFAMREL', 'PRFAMTYP', 'PEHSPNON', 'PRMARSTA', 'PRPERTYP', \
               'PENATVTY', 'PEMNTVTY', 'PEFNTVTY', 'PRCITSHP', 'PUSLFPRX', \
               'PEMLR', 'PUWK', 'PUBUS1', 'PUBUS2OT', 'PUBUSCK1', 'PUBUSCK2', \
               'PUBUSCK3', 'PUBUSCK4', 'PURETOT', 'PUDIS', 'PERET1', 'PUDIS1', \
               'PUDIS2', 'PUABSOT', 'PULAY', 'PEABSRSN', 'PEABSPDO', 'PEMJOT', \
               'PEHRFTPT', 'PEHRUSLT', 'PEHRWANT', 'PEHRRSN1', 'PEHRRSN2', 'PEHRRSN3', \
               'PUHROFF1', 'PUHROT1', 'PEHRAVL', 'PULAYDT', 'PULAY6M', 'PELAYAVL', \
               'PULAYAVR', 'PELAYLK', 'PELAYFTO', 'PULK', 'PELKM1', 'PULKM2', \
               'PULKM3', 'PULKM4', 'PULKM5', 'PULKM6', 'PULKDK1', 'PULKDK2', \
               'PULKPS1', 'PULKPS2', 'PULKPS3', 'PELKAVL', 'PULKAVR', 'PELKLL1O', \
               'PELKLL2O', 'PELKLWO', 'PELKFTO', 'PEDWWNTO', 'PEDWRSN', 'PEDWLKO', \
               'PEDWWK', 'PEDW4WK', 'PEDWLKWK', 'PEDWAVL', 'PEDWAVR', 'PUDWCK1', 'PUJHDP1O', 'PEJHRSN', 'PEJHWANT', 'PRABSREA', 'PRCIVLF', 'PRDISC', 'PREMPHRS', 'PREMPNOT', 'PREXPLF', 'PRFTLF', 'PRJOBSEA', 'PRPTREA', 'PRUNTYPE', 'PRWKSCH', 'PRWKSTAT', 'PRWNTJOB', 'PUJHCK3', 'PUJHCK3', 'PUJHCK4', 'PUJHCK5', 'PUIODP1', 'PUIODP2', 'PUIODP3', 'PEIO1COW', 'PUIO1MFG', 'PEIO2COW', 'PUIO2MFG', 'PUIOCK1', 'PUIOCK2', 'PRIOELG', 'PRAGNA', 'PRCOW1', 'PRCOW2', 'PRCOWPG', 'PRDTCOW1', 'PRDTCOW2', 'PRDTIND1', 'PRDTIND2', 'PRDTOCC1', 'PRDTOCC2', 'PREMP', 'PRMJIND1', 'PRMJIND2', 'PRMJOCC1', 'PRMJOCC2', 'PRMJOCGR', 'PRNAGPWS', 'PRNAGWS', 'PRSJMJ', 'PRERELG', 'PEERNUOT', 'PEERNPER', 'PEERNRT', 'PEERNHRY', 'PEERNLAB', 'PEERNCOV', 'PENLFJH', 'PENLFRET', 'PENLFACT', 'PESCHENR', 'PESCHFT', 'PESCHLVL', 'PRNLFSCH', 'PRWERNAL', 'PRHERNAL', 'HXTENURE', 'HXHOUSUT', 'HXHOUSUT', 'HXTELAVL', 'HXPHONEO', 'PXINUSYR', 'PXRRP', 'PXPARENT', 'PXAGE', 'PXMARITL', 'PXSPOUSE', 'PXSEX', 'PXAFWHN1', 'PXAFNOW', 'PXEDUCA', 'PXRACE1', 'PXNATVTY', 'PXMNTVTY', 'PXFNTVTY', 'PXHSPNON', 'PXMLR', 'PXRET1', 'PXABSRSN', 'PXABSPDO', 'PXMJOT', 'PXMJNUM', 'PXMJNUM', 'PXHRUSL2', 'PXHRFTPT', 'PXHRUSLT', 'PXHRWANT', 'PXHRRSN1', 'PXHRRSN2', 'PXHRACT1', 'PXHRACT2', 'PXHRACTT', 'PXHRRSN3', 'PXHRAVL', 'PXLAYAVL', 'PXLAYLK', 'PXLAYDUR', 'PXLAYFTO', 'PXLKM1', 'PXLKAVL', 'PXLKLL1O', 'PXLKLL2O', 'PXLKLWO', 'PXLKDUR', 'PXLKFTO', 'PXDWWNTO', 'PXDWRSN', 'PXDWLKO', 'PXDWWK', 'PXDW4WK', 'PXDWLKWK', 'PXDWAVL', 'PXDWAVR', 'PXJHWKO', 'PXJHRSN', 'PXJHWANT', 'PXIO1COW', 'PXIO1ICD', 'PXIO1OCD', 'PXIO2COW', 'PXIO2ICD', 'PXIO2OCD', 'PXERNUOT', 'PXERNPER', 'PXERNH1O', 'PXERNWKP', 'PXERNRT', 'PXERNHRY', 'PXERNH2', 'PXERNLAB', 'PXERNCOV', 'PXNLFJH', 'PXNLFRET', 'PXNLFACT', 'PXSCHENR', 'PXSCHFT', 'PXSCHFT', 'PEDIPGED', 'PEHGCOMP', 'PEHGCOMP', 'PEGR6COR', 'PEMS123', 'PXDIPGED', 'PXHGCOMP', 'PXCYC', 'PXGRPROF', 'PXGR6COR', 'PXMS123', 'PEIO1ICD', 'PEIO2ICD', 'PEIO2OCD', 'PRIMIND1', 'PRIMIND2', 'PEAFWHN1', 'PEAFWHN2', 'PEAFWHN3', 'PEAFWHN4', 'PXAFEVER', 'PELNDAD', 'PEDADTYP', 'PEMOMTYP', 'PXLNDAD', 'PXLNMOM', 'PXDADTYP', 'PXMOMTYP', 'PXCOHAB', 'PEDISEAR', 'PEDISEYE', 'PEDISREM', 'PEDISPHY', 'PEDISDRS', 'PEDISOUT', 'PRDISFLG', 'PXDISEAR', 'PXDISEYE', 'PXDISREM', 'PXDISPHY', 'PXDISDRS', 'PXDISOUT', 'PES1', 'PES2', 'PES3', 'PES4', 'PES5', 'PES6', 'PES7', 'PUSCK4']

In [126]:
#data_arr, categorical_idx = one_hot_encoding(data, categoricals)
#data_train_h2o=h2o.H2OFrame(data_arr)
#data_labels_h2o = h2o.H2OFrame(y_train)

real_data = pd.read_csv('test_2008.csv')
real_data = real_data.drop(columns=drop_columns)
real_data = real_data.drop(columns=['id'])
real_data += 1


combined_data = pd.concat([data, real_data])

combined_data, categorical_idx = one_hot_encoding(combined_data, categoricals)

train_data = combined_data[:64667]
train_data = np.column_stack((train_data, y_train[:]))
data_train_h2o = h2o.H2OFrame(train_data)

real_data = combined_data[64667:]
data_test_h2o=h2o.H2OFrame(real_data)

148


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [127]:
train_data.shape

(64667, 807)

In [128]:
data_train_h2o[-1]=data_labels_h2o[-1].asfactor()

In [52]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator
grid_search_gbm = H2OGradientBoostingEstimator(
    
  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 1.0,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 1.0,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 0
) 

ss = data_train_h2o.split_frame(seed = 1)
train = ss[0]
val = ss[1]

hyper_params2 = {
    'learn_rate':[0.01, 0.02],
    'max_depth':[2,4,6],
    'ntrees':[500, 1000, 5000]}

In [53]:
grid = H2OGridSearch(grid_search_gbm, hyper_params2,
                         grid_id='depth_grid',
                         search_criteria={'strategy': "Cartesian"})
#Train grid search
grid.train(x=data_train_h2o.names[:255], 
           y=data_train_h2o.names[255],
           training_frame=train,
           validation_frame=val)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [23]:
grid_sorted = grid.get_grid(sort_by='auc',decreasing=True)

In [38]:
print(grid_sorted.models[0].params['learn_rate'])

{'default': 0.1, 'actual': 0.02}


In [39]:
best_gbm = grid_sorted.models[0]
best_gbm.train(x=data_train_h2o.names[:255],\
            y=data_train_h2o.names[255], training_frame=data_train_h2o, model_id="GBM_Titanic")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [42]:
f=best_gbm.predict(test_data=data_test_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [43]:
f = f.as_data_frame()

In [47]:
preds = np.array(f['p1'])
process_for_submission(preds, 'h2o_gbm4')

16000


## Train an h2o gbm

In [129]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator


In [130]:
model = H2OGradientBoostingEstimator(## more trees is better if the learning rate is small enough 
  ## here, use "more than enough" trees - we have early stopping
  ntrees = 12000,                                                            

  ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
  learn_rate = 0.01,                                                         

  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 15, stopping_tolerance = 1e-5, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 1.0,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 1.0,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 0, max_depth=2)

In [ ]:
model.train(x=data_train_h2o.names[:-1],y=data_train_h2o.names[-1], training_frame=data_train_h2o, model_id="GBM_Titanic")

gbm Model Build progress: |██

In [118]:
model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_Titanic


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.14203152016362638
RMSE: 0.3768706942223372
LogLoss: 0.44080882675645516
Mean Per-Class Error: 0.264549825743557
AUC: 0.8120290074970485
pr_auc: 0.6180473637875367
Gini: 0.6240580149940971
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2946604691348905: 


,0,1,Error,Rate
0,37810.0,10342.0,0.2148,(10342.0/48152.0)
1,5280.0,11235.0,0.3197,(5280.0/16515.0)
Total,43090.0,21577.0,0.2416,(15622.0/64667.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2946605,0.5898876,223.0
max f2,0.1697967,0.7046475,295.0
max f0point5,0.4448034,0.5920652,154.0
max accuracy,0.4903028,0.7966815,136.0
max precision,0.9774875,1.0,0.0
max recall,0.0190711,1.0,397.0
max specificity,0.9774875,1.0,0.0
max absolute_mcc,0.3308409,0.4322752,204.0
max min_per_class_accuracy,0.2580014,0.7339882,242.0
max mean_per_class_accuracy,0.2661635,0.7354502,238.0


Gains/Lift Table: Avg response rate: 25.54 %, avg score: 25.54 %



...


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-02-09 22:04:14,0.003 sec,0.0,0.4360776,0.5681745,0.5,0.0,1.0,0.7446147
,2019-02-09 22:04:15,0.543 sec,1.0,0.4356344,0.5671600,0.6979383,0.2675724,2.0295912,0.3456477
,2019-02-09 22:04:15,0.816 sec,2.0,0.4351989,0.5661665,0.7083537,0.2705952,2.0295912,0.3856990
,2019-02-09 22:04:15,1.087 sec,3.0,0.4347717,0.5651948,0.7095462,0.2711334,2.0295912,0.3856990
,2019-02-09 22:04:16,1.334 sec,4.0,0.4343518,0.5642431,0.7083537,0.2705952,2.0295912,0.3856990
---,---,---,---,---,---,---,---,---,---
,2019-02-09 23:00:51,56 min 37.264 sec,10953.0,0.3769082,0.4408845,0.8119510,0.6179049,3.5525317,0.2417926
,2019-02-09 23:00:55,56 min 41.298 sec,10966.0,0.3768983,0.4408647,0.8119810,0.6179593,3.5525317,0.2418699
,2019-02-09 23:01:00,56 min 45.391 sec,10978.0,0.3768878,0.4408434,0.8120038,0.6179962,3.5525317,0.2421328
,2019-02-09 23:01:04,56 min 49.446 sec,10990.0,0.3768784,0.4408240,0.8120144,0.6180522,3.5525317,0.2429369



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
C720,53063.2226562,1.0,0.3472809
C718,20184.1601562,0.3803795,0.1320985
C8,9347.2666016,0.1761534,0.0611747
C721,5899.2480469,0.1111739,0.0386086
C716,4731.3969727,0.0891653,0.0309654
---,---,---,---
C772,0.0,0.0,0.0
C773,0.0,0.0,0.0
C788,0.0,0.0,0.0
C789,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [62]:
data_test_h2o=h2o.H2OFrame(real_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [119]:
f=model.predict(test_data=data_test_h2o)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [120]:
f = f.as_data_frame()

In [121]:
print(f)

       predict        p0        p1
0            0  0.791832  0.208168
1            0  0.924850  0.075150
2            0  0.849263  0.150737
3            1  0.499193  0.500807
4            0  0.778429  0.221571
5            0  0.925359  0.074641
6            1  0.232793  0.767207
7            1  0.240601  0.759399
8            0  0.966170  0.033830
9            0  0.742338  0.257662
10           0  0.963584  0.036416
11           1  0.510823  0.489177
12           0  0.708922  0.291078
13           0  0.727004  0.272996
14           0  0.761833  0.238167
15           0  0.905120  0.094880
16           1  0.628930  0.371070
17           0  0.958940  0.041060
18           0  0.819147  0.180853
19           1  0.669600  0.330400
20           1  0.684749  0.315251
21           0  0.858910  0.141090
22           0  0.932651  0.067349
23           0  0.949577  0.050423
24           0  0.829265  0.170735
25           0  0.968660  0.031340
26           0  0.970883  0.029117
27           1  0.54

In [122]:
preds = np.array(f['p1'])

In [123]:
process_for_submission(preds, 'h2o_gbm6')

16000


In [132]:
h2o_model1 = pd.read_csv('h2o_gbm3submission.csv')
h2o_model2 = pd.read_csv('h2o_gbm6submission.csv')

In [133]:
comb_h2o = pd.concat([h2o_model1, h2o_model2]).groupby(level=0).mean()

In [134]:
comb_h2o.to_csv('comb_h2o_submission.csv', index=None)

In [80]:
sk_h2o

,id,target
0,0,0.205410
1,1,0.076286
2,2,0.197070
3,3,0.418694
4,4,0.270673
5,5,0.086769
6,6,0.555657
7,7,0.738593
8,8,0.051482
9,9,0.271720
